In [ ]:
TIME_DIM = 4

import numpy as np
from keras.layers import MaxPooling3D, MaxPooling1D, Conv1D, RepeatVector, Lambda, Dot, Multiply, Input, Permute, LSTM, Dense, TimeDistributed, Conv2D, ConvLSTM2D, Flatten, Reshape, BatchNormalization
from keras.models import Sequential, Model
from keras.optimizers import adam
import keras.backend as k
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
from keras.utils import to_categorical

#d = np.random.rand(4000,18)
#lab = np.random.binomial(1, 0.2, size=d.shape[0])
d = pd.read_csv("Sandipan_compressed1.csv")
#d = np.fromfile("Sandipan_compressed.csv")
#d = d.values
#d = d.iloc[:, :-1]
d = d.values
print(d.shape)
lab = d[:,17]
d = d[:,0:17]
'''
d = d[:,:-1]
d = d.values
lab = d[:,20]
d = d[:,:19]
'''

x = np.zeros((d.shape[0]-TIME_DIM+1,TIME_DIM,d.shape[1]), dtype='float')
y = np.zeros((d.shape[0]-TIME_DIM+1,1), dtype='float')
SIZE = d.shape[0]-TIME_DIM+1

for i in range(SIZE):
    for j in range(TIME_DIM):
        x[i,j,:] = d[i+j,:]
    y[i] = lab[i+j]

print(x.shape)
print(np.unique(y))
y = to_categorical(y)
print(y.shape)

inp = Input(shape=(x.shape[1],x.shape[2]))
#*****Attention-1*************
perm1 = Permute((2,1))(inp)
att1 = Dense(TIME_DIM,activation='softmax')(perm1) 
perm2 = Permute((2,1))(att1)
merg1 = Multiply()([inp,perm2])
#*****END Attention-1*********

#*****Affinity Matrix*********
r_merg1 = Reshape((TIME_DIM,d.shape[1],1))(merg1)
r_perm2 = Reshape((TIME_DIM,1,d.shape[1]))(perm2)
mat_mul1 = Lambda(lambda x: tf.matmul(x[0],x[1]))([r_merg1,r_perm2])
r_mat_mul1 = Reshape((TIME_DIM,d.shape[1],d.shape[1],1))(mat_mul1)
#*****END Affinity Matrix*****

conv1 = ConvLSTM2D(64, 3, activation = 'selu',return_sequences=True)(r_mat_mul1)
bn1 = BatchNormalization()(conv1)
pool31 = MaxPooling3D((1,2,2))(bn1)
conv2 = ConvLSTM2D(128, 3, activation = 'selu',return_sequences=True)(pool31)
bn2 = BatchNormalization()(conv2)
pool32 = MaxPooling3D((1,2,2))(bn2)
conv3 = ConvLSTM2D(256, 2, activation = 'selu',return_sequences=True)(pool32)
bn3 = BatchNormalization()(conv3)
print(bn3.shape)
r_bn3 = Reshape((TIME_DIM,bn3.shape[2]*bn3.shape[3]*bn3.shape[4]))(bn3)
print(r_bn3.shape)

#*****Attention-2*************
perm3 = Permute((2,1))(r_bn3)
att2 = Dense(TIME_DIM,activation='softmax')(perm3)
mean1 = Lambda(lambda x: k.mean(x, axis=1))(att2)
rv = RepeatVector(256)(mean1)
perm4 = Permute((2,1))(rv)
merg2 = Multiply()([r_bn3,perm4])
#*****END Attention-2*********

mean2 = Lambda(lambda x: k.mean(x, axis=1))(merg2)
r_mean2 = Reshape((256,1))(mean2)
conv4 = Conv1D(256,3, activation = 'selu')(r_mean2)
bn4 = BatchNormalization()(conv4)
pool1 = MaxPooling1D(2)(bn4)
conv5 = Conv1D(512,3, activation = 'selu')(pool1)
bn5 = BatchNormalization()(conv5)
pool2 = MaxPooling1D(2)(bn5)
conv6 = Conv1D(512,3, activation = 'selu')(pool2)
bn6 = BatchNormalization()(conv6)
pool3 = MaxPooling1D(2)(bn6)
conv7 = Conv1D(512,3, activation = 'selu')(pool3)
bn7 = BatchNormalization()(conv7)
pool4 = MaxPooling1D(2)(bn7)
conv8 = Conv1D(512,3, activation = 'selu')(pool4)
bn8 = BatchNormalization()(conv8)
pool5 = MaxPooling1D(2)(bn8)
conv9 = Conv1D(1024,3, activation = 'selu')(pool5)
bn9 = BatchNormalization()(conv9)
pool6 = MaxPooling1D(2)(bn9)
flat = Flatten()(pool6)
dense2 = Dense(100, activation ='selu')(flat)
dense3 = Dense(y.shape[1], activation ='softmax')(dense2)
model = Model(inputs=inp,outputs=dense3)



model.summary()
model.compile(optimizer=adam(lr=0.00001), loss='categorical_crossentropy',metrics=['accuracy'])
model.fit(x,y,epochs=50,batch_size=32,validation_split=0.7)           # Change x to x_test, y to y_test
                                                  # Change epochs as required
y1 = model.predict(x)

model.evaluate(x,y)

#fig = plt.figure()
#plt.plot(y)
#plt.plot(y1)
#plt.show(fig)





